#Objective

## To visualize Confirmed,	Deaths and Recovered cases of top 10 countries most poulated countries in the world.

##To predict the future cases for 1 week for all these 10 countries.

In [1]:
#source: https://www.worldometers.info/world-population/population-by-country/
countries= ['India','China','US', 'Indonesia','Pakistan','Nigeria','Brazil','Bangladesh', 'Russia','Ethiopia']

##Important Libraries

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from prophet import Prophet
import matplotlib.pyplot as plt

In [3]:
df= pd.read_csv("/content/drive/MyDrive/intellipaat_assignments/COVID-19/covid_19_clean_complete.csv")

In [4]:
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [5]:
df.rename(columns= {'Country/Region': 'Country'}, inplace= True)

In [6]:
df.head()

,Province/State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


#Data Cleaning and Preprocessing

In [7]:
df.shape

(49068, 10)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  14664 non-null  object 
 1   Country         49068 non-null  object 
 2   Lat             49068 non-null  float64
 3   Long            49068 non-null  float64
 4   Date            49068 non-null  object 
 5   Confirmed       49068 non-null  int64  
 6   Deaths          49068 non-null  int64  
 7   Recovered       49068 non-null  int64  
 8   Active          49068 non-null  int64  
 9   WHO Region      49068 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 3.7+ MB


## All the columns datatypes are correct according to the values they hold except Date column. Let's change it to the DateTime format.


In [9]:
df['Date'].head()

,Date
0,2020-01-22
1,2020-01-22
2,2020-01-22
3,2020-01-22
4,2020-01-22


In [10]:
pd.to_datetime(df['Date'], yearfirst= True)

,Date
0,2020-01-22
1,2020-01-22
2,2020-01-22
3,2020-01-22
4,2020-01-22
...,...
49063,2020-07-27
49064,2020-07-27
49065,2020-07-27
49066,2020-07-27


In [11]:
df['Date']= pd.to_datetime(df['Date'], yearfirst= True)

In [12]:
df.head()

,Province/State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [13]:
df.isnull().sum()

,0
Province/State,34404
Country,0
Lat,0
Long,0
Date,0
Confirmed,0
Deaths,0
Recovered,0
Active,0
WHO Region,0


##We can drop the column Province/State as there are lot of null values in that column as well as it is not needed for the analysis.

In [14]:
df.drop('Province/State', axis=1 , inplace= True)

In [15]:
df.head()

,Country,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


## Let's create separate dataset for each countries and take Confirmed, Deaths, Recovered columns only.

In [16]:
#Dictionary to store country specific datasets
country_dfs= {}

for country in countries:
  condition=df['Country']==country
  country_dfs[country]= df[condition][['Date', 'Confirmed', 'Deaths', 'Recovered']]



In [17]:
country_dfs

{'India':             Date  Confirmed  Deaths  Recovered
 129   2020-01-22          0       0          0
 390   2020-01-23          0       0          0
 651   2020-01-24          0       0          0
 912   2020-01-25          0       0          0
 1173  2020-01-26          0       0          0
 ...          ...        ...     ...        ...
 47892 2020-07-23    1288108   30601     817209
 48153 2020-07-24    1337024   31358     849432
 48414 2020-07-25    1385635   32060     885573
 48675 2020-07-26    1435616   32771     917568
 48936 2020-07-27    1480073   33408     951166
 
 [188 rows x 4 columns],
 'China':             Date  Confirmed  Deaths  Recovered
 48    2020-01-22          1       0          0
 49    2020-01-22         14       0          0
 50    2020-01-22          6       0          0
 51    2020-01-22          1       0          0
 52    2020-01-22          0       0          0
 ...          ...        ...     ...        ...
 48883 2020-07-27        204       3       

In [18]:
country_dfs['India'].head()

,Date,Confirmed,Deaths,Recovered
129,2020-01-22,0,0,0
390,2020-01-23,0,0,0
651,2020-01-24,0,0,0
912,2020-01-25,0,0,0
1173,2020-01-26,0,0,0


In [19]:
country_dfs['India'].shape

(188, 4)

In [20]:
country_dfs['Brazil'].head()

,Date,Confirmed,Deaths,Recovered
28,2020-01-22,0,0,0
289,2020-01-23,0,0,0
550,2020-01-24,0,0,0
811,2020-01-25,0,0,0
1072,2020-01-26,0,0,0


In [21]:
country_dfs['Brazil'].shape

(188, 4)

#Visualization

In [22]:
country_dfs['India'].columns

Index(['Date', 'Confirmed', 'Deaths', 'Recovered'], dtype='object')

In [23]:
for country in countries:
  fig= px.line(
      country_dfs[country],
      x= 'Date',
      y= ['Confirmed', 'Deaths', 'Recovered'],
      title= f' Covid19 in {country} over time',
      labels= {'value': 'Count', 'variable': 'Type of Case'}
  )
  fig.show()


##From over graph we can observe that both in Us, Brazil and  India Covid 19 cases went over a million  over time.

In [24]:
df.columns

Index(['Country', 'Lat', 'Long', 'Date', 'Confirmed', 'Deaths', 'Recovered',
       'Active', 'WHO Region'],
      dtype='object')

In [25]:
deaths_df= df.groupby(['Country'], as_index= False)['Deaths'].sum()

In [26]:
deaths_df.head()

,Country,Deaths
0,Afghanistan,49098
1,Albania,5708
2,Algeria,77972
3,Andorra,5423
4,Angola,1078


In [27]:
deaths_df.shape

(187, 2)

In [28]:
deaths_df.sort_values(by= 'Deaths', ascending= False, inplace= True)

In [29]:
fig= px.bar(
    deaths_df[:20],
    x= 'Country',
    y= 'Deaths',
    title= 'Countries with the most Deaths',
    labels= {'Country/Region':'Country', 'Deaths': 'Death Counts'}
)
fig.show()

#Model Building

##Preparing Forecasting Data

In [30]:
forecast_data_dfs={}

for country in countries:
  country_dfs[country].rename(columns= {'Date':'ds', 'Confirmed':'y'},inplace = True)

  forecast_data_dfs[country]= country_dfs[country][['ds','y']]

In [31]:
forecast_data_dfs['India']

,ds,y
129,2020-01-22,0
390,2020-01-23,0
651,2020-01-24,0
912,2020-01-25,0
1173,2020-01-26,0
...,...,...
47892,2020-07-23,1288108
48153,2020-07-24,1337024
48414,2020-07-25,1385635
48675,2020-07-26,1435616


##Splitting the data into test and train.

In [32]:
train_dfs= {}
test_dfs={}

for country in countries:
  train_dfs[country]= forecast_data_dfs[country][:-7]
  test_dfs[country]= forecast_data_dfs[country][-7:]

In [33]:
train_dfs['India'].head()

,ds,y
129,2020-01-22,0
390,2020-01-23,0
651,2020-01-24,0
912,2020-01-25,0
1173,2020-01-26,0


In [34]:
train_dfs['India'].shape

(181, 2)

In [35]:
test_dfs['India'].shape

(7, 2)

In [36]:
forecast_dfs={}

for country in countries:
  model= Prophet()
  model.fit(train_dfs[country])
  future= model.make_future_dataframe(periods=7)
  forecast_dfs[country]= model.predict(future)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmprxf7d9jv/u_f21fsv.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmprxf7d9jv/7fxyty7m.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=13065', 'data', 'file=/tmp/tmprxf7d9jv/u_f21fsv.json', 'init=/tmp/tmprxf7d9jv/7fxyty7m.json', 'output', 'file=/tmp/tmprxf7d9jv/prophet_modelmkojkegu/prophet_model-20241229023853.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
02:38:53 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
02:38:53 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonalit

In [37]:
forecast_dfs['India']

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-22,-1.222163e+03,-2.833832e+04,2.110429e+04,-1.222163e+03,-1.222163e+03,-1656.242715,-1656.242715,-1656.242715,-1656.242715,-1656.242715,-1656.242715,0.0,0.0,0.0,-2.878405e+03
1,2020-01-23,-1.187653e+03,-2.560130e+04,2.246081e+04,-1.187653e+03,-1.187653e+03,-1007.246775,-1007.246775,-1007.246775,-1007.246775,-1007.246775,-1007.246775,0.0,0.0,0.0,-2.194900e+03
2,2020-01-24,-1.153143e+03,-2.603774e+04,2.251790e+04,-1.153143e+03,-1.153143e+03,-222.601835,-222.601835,-222.601835,-222.601835,-222.601835,-222.601835,0.0,0.0,0.0,-1.375745e+03
3,2020-01-25,-1.118633e+03,-2.507422e+04,2.262670e+04,-1.118633e+03,-1.118633e+03,1004.473660,1004.473660,1004.473660,1004.473660,1004.473660,1004.473660,0.0,0.0,0.0,-1.141591e+02
4,2020-01-26,-1.084123e+03,-2.344415e+04,2.536810e+04,-1.084123e+03,-1.084123e+03,1777.297278,1777.297278,1777.297278,1777.297278,1777.297278,1777.297278,0.0,0.0,0.0,6.931746e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2020-07-23,1.114476e+06,1.088679e+06,1.137435e+06,1.114001e+06,1.115038e+06,-1007.246775,-1007.246775,-1007.246775,-1007.246775,-1007.246775,-1007.246775,0.0,0.0,0.0,1.113469e+06
184,2020-07-24,1.135539e+06,1.110883e+06,1.160569e+06,1.134522e+06,1.136679e+06,-222.601835,-222.601835,-222.601835,-222.601835,-222.601835,-222.601835,0.0,0.0,0.0,1.135317e+06
185,2020-07-25,1.156602e+06,1.132283e+06,1.183958e+06,1.154859e+06,1.158505e+06,1004.473660,1004.473660,1004.473660,1004.473660,1004.473660,1004.473660,0.0,0.0,0.0,1.157607e+06
186,2020-07-26,1.177665e+06,1.154905e+06,1.205521e+06,1.174856e+06,1.180799e+06,1777.297278,1777.297278,1777.297278,1777.297278,1777.297278,1777.297278,0.0,0.0,0.0,1.179443e+06


##Merging Forecast and predicted values to calculate error

In [38]:
#Merging Test and forecast data for last 7 days
merged_dfs={}

for  country in countries:
  forecast_dfs[country]= forecast_dfs[country][['ds', 'yhat']]
  merged_dfs[country]= pd.merge(forecast_dfs[country], test_dfs[country], on= 'ds', how= 'inner')

In [39]:
merged_dfs['India']

,ds,yhat,y
0,2020-07-21,1.070283e+06,1193078
1,2020-07-22,1.091757e+06,1238798
2,2020-07-23,1.113469e+06,1288108
3,2020-07-24,1.135317e+06,1337024
4,2020-07-25,1.157607e+06,1385635
5,2020-07-26,1.179443e+06,1435616
6,2020-07-27,1.200900e+06,1480073


##Visualizing Actual_values with Predicted_Values

In [40]:
for country in countries:
  fig= px.line(
      merged_dfs[country],
      x= 'ds',
      y= ['y','yhat'],
      title= f' {country}: Actual vs Predicted Confirmed Cases',
  )
  fig.show()


##Except for China, model is  predicting well but let's check it with the metrics

In [41]:
# Calculating Mean Absolute Percentage Error

for country in countries:
  mape= np.mean(np.abs(( merged_dfs[country]['y']- merged_dfs[country]['yhat'])/merged_dfs[country]['y']))*100
  print(f' {country}: Prediction error= {mape} %')


 India: Prediction error= 14.852699368994982 %
 China: Prediction error= 38224.81231399238 %
 US: Prediction error= 6.487826644856202 %
 Indonesia: Prediction error= 4.278237250716996 %
 Pakistan: Prediction error= 9.720995378108393 %
 Nigeria: Prediction error= 0.46487769394999295 %
 Brazil: Prediction error= 2.048278081349672 %
 Bangladesh: Prediction error= 2.8903141849204745 %
 Russia: Prediction error= 1.2360412186194296 %
 Ethiopia: Prediction error= 23.851103597171655 %


## Model's performance is best for Nigeria and worst for China and acceptable for other countries

#Predict the future of 7 days

In [42]:
forecast_dfs={}

for country in countries:
  model= Prophet()
  model.fit(forecast_data_dfs[country])
  future= model.make_future_dataframe(periods=7)
  forecast_dfs[country]= model.predict(future)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmprxf7d9jv/4358navq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmprxf7d9jv/2b1o0g13.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97389', 'data', 'file=/tmp/tmprxf7d9jv/4358navq.json', 'init=/tmp/tmprxf7d9jv/2b1o0g13.json', 'output', 'file=/tmp/tmprxf7d9jv/prophet_model4p8iiopd/prophet_model-20241229023858.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
02:38:58 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
02:38:58 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonalit

In [43]:
forecast_dfs['India']

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-22,-2.103805e+03,-3.837321e+04,3.092957e+04,-2.103805e+03,-2.103805e+03,-2115.615216,-2115.615216,-2115.615216,-2115.615216,-2115.615216,-2115.615216,0.0,0.0,0.0,-4.219421e+03
1,2020-01-23,-2.040100e+03,-3.596842e+04,3.168973e+04,-2.040100e+03,-2.040100e+03,-1031.630122,-1031.630122,-1031.630122,-1031.630122,-1031.630122,-1031.630122,0.0,0.0,0.0,-3.071730e+03
2,2020-01-24,-1.976395e+03,-3.717340e+04,3.307123e+04,-1.976395e+03,-1.976395e+03,179.948267,179.948267,179.948267,179.948267,179.948267,179.948267,0.0,0.0,0.0,-1.796446e+03
3,2020-01-25,-1.912689e+03,-3.609013e+04,3.552597e+04,-1.912689e+03,-1.912689e+03,1117.586848,1117.586848,1117.586848,1117.586848,1117.586848,1117.586848,0.0,0.0,0.0,-7.951022e+02
4,2020-01-26,-1.848984e+03,-3.712018e+04,3.614021e+04,-1.848984e+03,-1.848984e+03,2195.349076,2195.349076,2195.349076,2195.349076,2195.349076,2195.349076,0.0,0.0,0.0,3.463654e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2020-07-30,1.410286e+06,1.375000e+06,1.441091e+06,1.409229e+06,1.411172e+06,-1031.630122,-1031.630122,-1031.630122,-1031.630122,-1031.630122,-1031.630122,0.0,0.0,0.0,1.409255e+06
191,2020-07-31,1.436538e+06,1.403570e+06,1.469761e+06,1.434670e+06,1.438289e+06,179.948267,179.948267,179.948267,179.948267,179.948267,179.948267,0.0,0.0,0.0,1.436718e+06
192,2020-08-01,1.462789e+06,1.427099e+06,1.497904e+06,1.459567e+06,1.465538e+06,1117.586848,1117.586848,1117.586848,1117.586848,1117.586848,1117.586848,0.0,0.0,0.0,1.463907e+06
193,2020-08-02,1.489041e+06,1.457317e+06,1.523549e+06,1.484568e+06,1.493033e+06,2195.349076,2195.349076,2195.349076,2195.349076,2195.349076,2195.349076,0.0,0.0,0.0,1.491236e+06


In [44]:
#Merging Test and forecast data for next 7 days
merged_dfs={}

for  country in countries:
  forecast_dfs[country]= forecast_dfs[country][['ds', 'yhat']]
  merged_dfs[country]= pd.merge(forecast_data_dfs[country], forecast_dfs[country], on= 'ds', how= 'outer')

In [45]:
for country in countries:
  fig= px.line(
      merged_dfs[country],
      x= 'ds',
      y= ['y','yhat'],
      title= f' {country}: Confirmed cases- Future predictions and Actual values',
  )
  fig.show()